# **Mount the Google drive**
The trained models are uploaded to the google drive and inorder to load them the drive is mounted.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/manikanta-varaganti/detection-of-motorcyclists-without-helmet

Cloning into 'detection-of-motorcyclists-without-helmet'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 46 (delta 0), reused 43 (delta 0), pack-reused 0
Receiving objects: 100% (46/46), 29.49 MiB | 37.37 MiB/s, done.
Filtering content: 100% (4/4), 860.43 MiB | 83.32 MiB/s, done.


In [3]:
%cd /content/detection-of-motorcyclists-without-helmet/HelmetDetection

/content/detection-of-motorcyclists-without-helmet/HelmetDetection


# **Import the required packages**

In [4]:
# clone Tensorflow object detection api
!git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 96720, done.
remote: Total 96720 (delta 0), reused 0 (delta 0), pack-reused 96720
Receiving objects: 100% (96720/96720), 610.81 MiB | 41.71 MiB/s, done.
Resolving deltas: 100% (70418/70418), done.


In [5]:
# Run to install proto buffers for object detection api
!apt-get update
!apt-get install -y -qq protobuf-compiler python-pil python-lxml

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [773 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,356 kB]
Get:13 http://archive.ubuntu.com/ubun

In [6]:
pip install inference-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00


In [7]:
import sys
sys.path.append('/content/detection-of-motorcyclists-without-helmet/HelmetDetection/models/research/slim')
!pip install tf_slim

In [8]:
%cd /content/detection-of-motorcyclists-without-helmet/HelmetDetection/models/research
!protoc object_detection/protos/*.proto --python_out=.


/content/detection-of-motorcyclists-without-helmet/HelmetDetection/models/research


In [9]:
# Import packages
import os
import cv2
import numpy as np
import tensorflow as tf
import sys
from google.colab.patches import cv2_imshow
from inference_sdk import InferenceHTTPClient
from PIL import Image
# Import utilites
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [10]:
pwd

'/content/detection-of-motorcyclists-without-helmet/HelmetDetection/models/research'

# **Configure the paths to trained models**
A Faster RCNN model is trained to detect motorcyclists in a given image.

A YOLO model is trained to detect helmet in the given image of motorcyclist.



In [11]:
CWD_PATH = '/content/detection-of-motorcyclists-without-helmet/HelmetDetection/'

MODEL_RCNN = 'rcnn'

MODEL_YOLO='yolo'

# Path to frozen detection graph .pb file, which contains the model that is used
# for rcnn object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_RCNN,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,MODEL_RCNN,'label_map.pbtxt')

# Path to yolo files
configPath=os.path.join(CWD_PATH,MODEL_YOLO,'yolov3_custom.cfg')
weightsPath=os.path.join(CWD_PATH,MODEL_YOLO,'yolov3_custom_4000.weights')
labelsPath=os.path.join(CWD_PATH,MODEL_YOLO,'obj.names')


# **Configure the input and output paths**

Images of Motorcyclists without helmet are stored in the Output folder.


In [12]:
# Path to input image
IMAGE_NAME = 'input/images/sample.jpeg'

# Path to input video
VIDEO_NAME = 'input/videos/clip.mp4'

OUTPUT_FOLDER='output/'

# Path to output
PATH_TO_OUTPUT = os.path.join(CWD_PATH, OUTPUT_FOLDER,'/images')
VIDEO_OUTPUT = os.path.join(CWD_PATH, OUTPUT_FOLDER,'output_clip.mp4')
PATH_TO_IMAGE = os.path.join(CWD_PATH,IMAGE_NAME)
NUMBER_PLATE_IMG_FOLDER = 'Number_plate_img/'
PATH_TO_NUMBER_PLATE_IMG_FOLDER = os.path.join(CWD_PATH, NUMBER_PLATE_IMG_FOLDER)
# Path to video
PATH_TO_VIDEO = os.path.join(CWD_PATH,VIDEO_NAME)

# **Configure Faster RCNN Model**

In [13]:

# Load the label map.
# Label maps map indices to category names
# Here we use internal utility functions
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=1, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
    sess = tf.compat.v1.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

In [14]:
categories

[{'id': 1, 'name': 'bikerider'}]

# **Configure YOLO Model**

In [15]:
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
ln = net.getLayerNames()

# Get the indices of the output layers
output_layer_indices = net.getUnconnectedOutLayers()

# Ensure output_layer_indices is in the correct format
if not isinstance(output_layer_indices[0], list):
    output_layer_indices = [output_layer_indices]

# Retrieve the names of the output layers using the indices
ln = [ln[i[0] - 1] for i in output_layer_indices]
LABELS = open(labelsPath).read().strip().split("\n")

In [16]:
LABELS

['Helmet', 'No Helmet']

# **Detection Of Motorcyclists Without Helmet From Input Image**
The two models are integrated to detect the motorcyclists without helmet from the input image and the images of riders without helmet are stored in the output folder

In [17]:
pip install imutils

In [18]:
pip install tensorflow

# **Detection Of Motorcyclists Without Helmet From Input Video Stream**
The two models are integrated to detect the motorcyclists without helmet from the input video stream and the images of riders without helmet are stored in the output folder, along with the resulting video stream

In [19]:
pip install keras-ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 973.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 6.9 MB/s eta 0:00:00


In [20]:
import cv2

def slow_down_video(input_video_path, output_video_path, slow_factor):
    # Open the input video file
    video_capture = cv2.VideoCapture(input_video_path)

    # Get video properties
    frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_video = cv2.VideoWriter(output_video_path, fourcc, fps / slow_factor, (frame_width, frame_height))

    # Read and write frames
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        # Write each frame 'slow_factor' times
        for _ in range(slow_factor):
            output_video.write(frame)

    # Release video capture and writer
    video_capture.release()
    output_video.release()
    cv2.destroyAllWindows()

# Example usage:
input_video_path = '/content/Swathi.mp4'
output_video_path = '/content/Swathi_slowed.mp4'
slow_factor = 2  # The factor by which to slow down the video
slow_down_video(input_video_path, output_video_path, slow_factor)

In [ ]:
import imutils
import shutil
from inference_sdk import InferenceHTTPClient
from google.colab.patches import cv2_imshow
import cv2
from PIL import Image
import platform  # Import the platform module
import matplotlib.pyplot as plt
import keras_ocr
import numpy as np
#video = cv2.VideoCapture("PATH_TO_VIDEO")
video_name = '/content/video4.mp4'
video = cv2.VideoCapture('/content/video4.mp4')
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="FjebqrmWpFtuakWjQmwC"
)

images=[]
f=0
size = (0, 0)
j = 0
while(video.isOpened()):
  if f%3!=0:
    ret,image = video.read()
    f+=1
    continue
  ret,image = video.read()
  if ret == True:
    image=cv2.resize(image,None,fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    image_expanded = np.expand_dims(image, axis=0)

    # Perform the actual detection by running the model with the image as input
    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: image_expanded})


    #getting the normalized coordinates of boxes
    normalizedBoxes = np.squeeze(boxes)
    normalizedScores = np.squeeze(scores)
    normalizedClasses = np.squeeze(classes)

    #set a min thresh score, say 0.8
    min_score_thresh = 0.8
    detectedBoxes = normalizedBoxes[normalizedScores > min_score_thresh]
    detectedClasses= normalizedClasses[normalizedScores > min_score_thresh]

    #get image size
    im_height,im_width, _=image.shape
    size=(im_width, im_height)

    #get coordinates of bike riders in image
    final_boxes = []
    for i in range(len(detectedBoxes)):
        ymin, xmin, ymax, xmax = detectedBoxes[i]
        final_boxes.append([xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height])

    j=0
    for [left,right,top,bottom] in final_boxes:
        l=int(round(left))
        r=int(round(right))
        t=int(round(top))
        b=int(round(bottom))
        croppedImage=image[t:b,l:r]  # Extract each bike rider

        cv2.rectangle(image, (l,t), (r,b), (255,0,0), 2)

        # perform padding to cropped image
        rows = croppedImage.shape[0]
        cols = croppedImage.shape[1]
        padding=0
        if rows > cols:
            padding = int((rows-cols) / 2)
            paddedImg=cv2.copyMakeBorder(croppedImage, 0,0,padding, padding,  cv2.BORDER_CONSTANT, (0, 0, 0))
        else:
            paddedImg=croppedImage

        # image preprocessing for yolo model
        (H, W) = paddedImg.shape[:2]
        blob = cv2.dnn.blobFromImage(paddedImg, 1 / 255.0, (416, 416), swapRB=True, crop=False)

        # run yolo model
        net.setInput(blob)
        layerOutputs = net.forward(ln)

        # Initializing for getting box coordinates, confidences, classid
        boxesH = []
        confidencesH = []
        classIDsH = []
        thresholdH = 0.15

        # getting coordinates of output predictions and performing NMS
        for output in layerOutputs:
            for detection in output:
                confidenceOfEachClass = detection[5:]
                classIDH = np.argmax(confidenceOfEachClass)    # get class with max confidence
                confidenceH = confidenceOfEachClass[classIDH]  # get the max confidence
                if confidenceH > thresholdH:
                    boxH = detection[0:4] * np.array([W, H, W, H])
                    (centerX, centerY, widthH, heightH) = boxH.astype("int")
                    x = int(centerX - (widthH / 2))
                    y = int(centerY - (heightH / 2))
                    boxesH.append([x, y, int(widthH), int(heightH)])
                    confidencesH.append(float(confidenceH))
                    classIDsH.append(classIDH)
        idxs = cv2.dnn.NMSBoxes(boxesH, confidencesH, thresholdH, 0.1)

        # bounding boxes for helmet- no helmet
        if len(idxs) > 0:
            for i in idxs.flatten():
                (x, y) = (boxesH[i][0], boxesH[i][1])
                (w, h) = (boxesH[i][2], boxesH[i][3])
                if LABELS[classIDsH[i]] == 'Helmet':
                    color = (0, 255, 0)
                    cv2.rectangle(image, (x-padding, y), (x + w-padding, y + h), color, 2)
                    text = "{}".format(LABELS[classIDsH[i]])
                    cv2.putText(image, text, (x//2, y+ 4*h//3),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)
                if (LABELS[classIDsH[i]] == 'No Helmet'):
                    #store images with no helmet
                    name=VIDEO_NAME.split(".")[0].split('/')[-1]+"_"+str(f)+"_"
                    j+=1
                    cv2.imwrite(PATH_TO_OUTPUT + '{}.jpg'.format(name+str(j)),croppedImage)
                    # draw bounding boxes
                    color = (0, 0, 255)
                    cv2.rectangle(image, (x-padding, y), (x + w-padding, y + h), color, 2)
                    text = "{}".format(LABELS[classIDsH[i]])
                    cv2.putText(image, text, (x//2, y + 4* h//3),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)
                    #Extract Number Plate
                    result = CLIENT.infer(croppedImage, model_id="vehicle-registration-plates-trudk/2")
                    # Check if the result contains predictions
                    if len(result['predictions']) > 0:
                      cv2_imshow(croppedImage)
                      predictions = result['predictions']
                      image = croppedImage
                      # Draw bounding boxes on the image
                      for prediction in predictions:
                          class_name = prediction.get('class')
                          confidence = prediction.get('confidence')
                          x, y, width, height = prediction.get('x'), prediction.get('y'), prediction.get('width'), prediction.get('height')

                          # Convert coordinates to integers
                          x, y, width, height = int(x), int(y), int(width), int(height)
                          xmax, ymax = int(x + width), int(y + height)
                          xmin, xmax, ymin, ymax = x - (width//2), x + (width//2), y - (width//2), y + (width//2)
                          # Draw bounding box
                          cv2.rectangle(croppedImage, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                          #cv2.putText(croppedImage, f'{class_name} {confidence:.2f}', (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                          number_plate = croppedImage[ymin:ymax, xmin: xmax]
                      # Show the image with bounding boxes
                      cv2_imshow(image)
                      cv2_imshow(number_plate)
                      pipeline = keras_ocr.pipeline.Pipeline()

                      # Path to your image file
                      #image_path = '<path_to_image_file>'

                      # Read the image
                      images = [keras_ocr.tools.read(number_plate)]

                      # Perform OCR on the image
                      prediction_groups = pipeline.recognize(images)

                      # Extract characters from the predictions
                      license_plate = ""
                      for image, predictions in zip(images, prediction_groups):
                          for text, box in predictions:
                              # Extracted character
                              character = text
                              # Bounding box coordinates
                              x_min, y_min, x_max, y_max = box
                              # Optionally, you can draw bounding boxes on the image
                              keras_ocr.tools.drawAnnotations(image=image, predictions=[(text, box)])
                              # Print the extracted character
                              print("Character:", character)
                              license_plate += character

                      # Show the image with bounding boxes (optional)
                      plt.imshow(images[0])
                      plt.axis('off')
                      plt.show()
                      print(license_plate)

                    else:
                      print("Number plate not found")
        images.append(image)
    f+=1
    #cv2_imshow(image)
  else:
    break
video.release()

out = cv2.VideoWriter(VIDEO_OUTPUT,cv2.VideoWriter_fourcc(*'mp4v'), 15, size)
for i in range(len(images)):
    out.write(images[i])
out.release()